In [1]:
import tensorflow as tf
import os
import numpy as np
import pickle

CIFAR_DIR = "cifar-10-batches-py"

In [2]:
def load_data(fileName):
    """从文件中读取数据"""
    with open(fileName, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        # 将数据缩放到[-1, 1]
        self._data = self._data
        self._labels = np.hstack(all_labels)
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        
        self._indicator = 0  # 当前遍历到的位置
        if self._need_shuffle:
            self._shuffle_data()
        
        
    """打乱训练数据集"""
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    """返回batch_size个样本"""
    def next_batch (self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch_size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    
"""加载数据"""  
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)





(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
batch_size = 20
x = tf.placeholder(tf.float32, [batch_size, 3072])
y = tf.placeholder(tf.int64, [batch_size])

In [4]:
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32 * 32
x_image = tf.transpose(x_image,  perm=[0, 2, 3, 1])

x_image_arr = tf.split(x_image, num_or_size_splits=batch_size, axis=0)
result_x_image_arr = []

for x_single_image in x_image_arr:
    x_single_image = tf.reshape(x_single_image, [32, 32, 3])
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=63)
    data_aug_3 = tf.image.random_contrast(data_aug_2, lower=0.2, upper=1.8)
    x_single_image = tf.reshape(data_aug_3, [1, 32, 32, 3])
    result_x_image_arr.append(x_single_image)
result_x_image = tf.concat(result_x_image_arr, axis=0)

normal_result_x_image = result_x_image / 127.5 - 1

conv1_1 = tf.layers.conv2d(normal_result_x_image, 
                           32, # 卷积核输出的通道数
                           (3, 3), #卷积核的大小
                           padding='same', #使用padding 使得图片输出大小相等
                           activation=tf.nn.relu, 
                           name='conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1, 
                           32, # 卷积核输出的通道数
                           (3, 3), #卷积核的大小
                           padding='same', #使用padding 使得图片输出大小相等
                           activation=tf.nn.relu, 
                           name='conv1_2')

# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2, 
                                  (2, 2), # 核大小
                                  (2, 2), # 步长
                                  name='pool1')

conv2_1 = tf.layers.conv2d(pooling1, 
                           32, # 卷积核输出的通道数
                           (3, 3), #卷积核的大小
                           padding='same', #使用padding 使得图片输出大小相等
                           activation=tf.nn.relu, 
                           name='conv2_1')

conv2_2 = tf.layers.conv2d(conv2_1, 
                           32, # 卷积核输出的通道数
                           (3, 3), #卷积核的大小
                           padding='same', #使用padding 使得图片输出大小相等
                           activation=tf.nn.relu, 
                           name='conv2_2')

# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2, 
                                  (2, 2), # 核大小
                                  (2, 2), # 步长
                                  name='pool2')

conv3_1 = tf.layers.conv2d(pooling2, 
                           32, # 卷积核输出的通道数
                           (3, 3), #卷积核的大小
                           padding='same', #使用padding 使得图片输出大小相等
                           activation=tf.nn.relu, 
                           name='conv3_1')

conv3_2 = tf.layers.conv2d(conv3_1, 
                           32, # 卷积核输出的通道数
                           (3, 3), #卷积核的大小
                           padding='same', #使用padding 使得图片输出大小相等
                           activation=tf.nn.relu, 
                           name='conv3_2')

#4 * 4
pooling3 = tf.layers.max_pooling2d(conv3_2, 
                                  (2, 2), # 核大小
                                  (2, 2), # 步长
                                  name='pool3')

#[None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)

y_ = tf.layers.dense(flatten, 10)

"""交叉熵损失函数"""
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_


#indices
predict = tf.arg_max(y_, 1)
corrent_preicttion = tf.equal(predict, y)
#准确率
accuracy = tf.reduce_mean(tf.cast(corrent_preicttion, tf.float64))


#梯度下降
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use `argmax` instead


In [5]:
# 制定面板图上显示的变量
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

inputs_summary = tf.summary.image('inputs_image', result_x_image)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])


# tensorboard 创建文件夹
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [6]:
# 初始化函数
init = tf.global_variables_initializer() 
batch_steps = 10000
test_steps = 100

output_summary_every_steps = 100

#实例化session
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    
    for i in range(batch_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps) == 0
        if should_output_summary:
            eval_ops.append(merged_summary)
        
        eval_ops_results = sess.run(eval_ops, feed_dict={x: batch_data, y: batch_labels})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test], feed_dict={x: fixed_test_batch_data, y: fixed_test_batch_labels})[0]
            test_writer.add_summary(test_summary_str, i+1)
        
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
#         每5000次进行一次测试
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' %(i+1, test_acc))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1_1/Conv2D (defined at <ipython-input-4-a8993fd400a3>:24)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](train_op/gradients/conv1_1/Conv2D_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv1_1/kernel/read)]]

Caused by op 'conv1_1/Conv2D', defined at:
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\site-packa